<a href="https://colab.research.google.com/github/rsha0812/EVA4-S4-Assignment/blob/master/S4_Assignment_of_EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        nn.ReLU(),
        nn.BatchNorm2d(16),
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
        nn.ReLU(),
        nn.BatchNorm2d(16),
        self.pool1 = nn.MaxPool2d(2, 2)
        nn.Dropout(0.25)
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1)
        nn.ReLU(),
        nn.BatchNorm2d(16),
        self.conv4 = nn.Conv2d(16, 20, 3, padding=1)
        nn.ReLU(),
        nn.BatchNorm2d(20),
        self.pool2 = nn.MaxPool2d(2, 2)
        nn.Dropout(0.2)
        self.conv5 = nn.Conv2d(20, 20, 3)
        nn.ReLU(),
        nn.BatchNorm2d(20),
        self.conv6 = nn.Conv2d(20, 32, 3)
        nn.ReLU(),
        nn.BatchNorm2d(32),
        self.conv7 = nn.Conv2d(32, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [15]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
            Conv2d-2           [-1, 16, 28, 28]           2,320
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 16, 14, 14]           2,320
            Conv2d-5           [-1, 20, 14, 14]           2,900
         MaxPool2d-6             [-1, 20, 7, 7]               0
            Conv2d-7             [-1, 20, 5, 5]           3,620
            Conv2d-8             [-1, 32, 3, 3]           5,792
            Conv2d-9             [-1, 10, 1, 1]           2,890
Total params: 20,002
Trainable params: 20,002
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.28
Params size (MB): 0.08
Estimated Total Size (MB): 0.36
---------------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 32
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [18]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.2825, Accuracy: 8907/10000 (89.07%)



  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.2552, Accuracy: 8978/10000 (89.78%)



  0%|          | 0/1875 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0409, Accuracy: 9919/10000 (99.19%)



  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0347, Accuracy: 9906/10000 (99.06%)



loss=0.005916237831115723 batch_id=1874: 100%|██████████| 1875/1875 [00:21<00:00, 87.33it/s]



Test set: Average loss: 0.0370, Accuracy: 9894/10000 (98.94%)

